# Evolutionary algorithms analysis



In [ ]:
# Ev algorithms setup

# Imports
import source.visualization as vis
import source.evolutionary_algorithms as ea
import importlib
import inspect
import sys
import os

# Load benchmark functions
module_path = os.path.abspath(os.path.join('./source'))
sys.path.append(module_path)
benchmarks = importlib.import_module('benchmark_functions')
functions = [obj for name, obj in inspect.getmembers(benchmarks) if inspect.isfunction(obj)]

# Hyperparameters
BOUNDS = (-100, 100)
DIMENSIONS = [2, 10, 30]
POPULATION_SIZES = [10, 20, 50]
FUNCTION_EVALUATIONS = 2000
RESULTS = []
REPEATS = 30

## DE

In [ ]:
def run_ev():
    for i, dim in enumerate(DIMENSIONS):
        pop_size = POPULATION_SIZES[i]
        evaluations = dim * FUNCTION_EVALUATIONS

        for function in functions:
            for j in range(REPEATS):
                # run DE Rand 1
                a = ea.differential_evolution(function, BOUNDS, pop_size, strategy="rand/1/bin")
                # run DE Best 1
                b = ea.differential_evolution(function, BOUNDS, pop_size, strategy="best/1/bin")
                # run PSO
                c = ea.particle_swarm_optimization(function, pop_size, dim, evaluations)
                # run SOMA-all-to-one
                d = ea.soma_all_to_one(function, BOUNDS, pop_size, evaluations)
                # run SOMA-all-to-all
                e = ea.soma_all_to_all(function, BOUNDS, pop_size, evaluations)
